<a href="https://colab.research.google.com/github/lathadharmalingam/my_coding/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install SHAPE PREDICTOR


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install playsound


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from scipy.spatial import distance as dist
from imutils import face_utils
from threading import Thread
import playsound
import argparse
import dlib
import cv2

def sound_alarm(path):
	playsound.playsound(path)

def eye_aspect_ratio(eye):
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	C = dist.euclidean(eye[0], eye[3])

	ear = (A + B) / (2.0 * C)
	return ear

ap = argparse.ArgumentParser()
ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
ap.add_argument("-a", "--alarm", type=str, default="", help="https://drive.google.com/file/d/1V9boJbqZXA6zydr5ZYdolSiL1LXGYjsV/view?usp=sharing")
ap.add_argument("-s", "--sms", type=str, default="", help= "+91 9150842667")
ap.add_argument("-m", "--mail", type=str, default="", help="lathadharmanlingam@gmail.com")
args = vars(ap.parse_args())

EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 28
MSG_COUNT_THRESH = 1

COUNTER = 0
ALARM_ON = False
MSG_COUNTER = 0

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

print("\n[ INFO:0] Loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

print("[ INFO:1] Starting video streaming...")
cap = cv2.VideoCapture(0)

while True:
	frame = cap.read()[1]
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)

	for (x,y,w,h) in faces:
		cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
		cv2.putText(frame,'FACE',(x+w//2-9,y+h+9), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (11,255,255), 2, cv2.LINE_AA)

	rects = detector(gray, 0)
	for rect in rects:
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)

		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]
		leftEAR = eye_aspect_ratio(leftEye)
		rightEAR = eye_aspect_ratio(rightEye)

		ear = (leftEAR + rightEAR) / 2.0

		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

		if ear < EYE_AR_THRESH:
			COUNTER += 1

			if COUNTER >= EYE_AR_CONSEC_FRAMES:
				if not ALARM_ON:
					ALARM_ON = True
					MSG_COUNTER += 1

					if args["alarm"] != "":
						t = Thread(target=sound_alarm, args=(args["alarm"],))
						t.deamon = True
						t.start()

				cv2.putText(frame, "DROWSINESS ALERT!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

		else:
			COUNTER = 0
			ALARM_ON = False

		cv2.putText(frame, "E.A.R. - {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		
		if MSG_COUNTER > MSG_COUNT_THRESH:
			import location
			if args["sms"] != "":
				import sms
				sms.send_sms(args["sms"], location.website, location.co_ordinates)
			else:
				print("\nUnable to send SMS, plz provide a registered mobile number.")
			if args["mail"] !="":
				import mail
				mail.send_mail(args["mail"], location.website, location.co_ordinates, location.msg_distance)
			else:
				print("\nUnable to send Email, plz provide an email id.")
			MSG_COUNTER = 0
 
	cv2.imshow("Frame", frame)
	
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
	   break

cap.release()
cv2.destroyAllWindows()

usage: ipykernel_launcher.py [-h] -p SHAPE_PREDICTOR [-a ALARM] [-s SMS]
                             [-m MAIL]
ipykernel_launcher.py: error: the following arguments are required: -p/--shape-predictor


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
pip install geocoder


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.3 MB/s 
     |████████████████████████████████| 127 kB 44.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from math import radians, cos, sin, asin, sqrt
import geocoder
import json
import requests

"""Driver details"""
user_latitude = 13.008826088580236, 
user_longitude = user_latitude =  80.00333002185538

city = "chennai"
print("\nUser/Driver details:")
print ("latitude = " + str(user_latitude) + "\nlongitude = " + str(user_longitude) + "\ncity = " + city)

"""Emergency contact person details"""
#visit https://stackoverflow.com/questions/24906833/get-your-location-through-python/49778452#49778452 to known about, how to get your own access_key
freegeoip = "http://api.ipstack.com/check?access_key=c3534305991e0cffdb31c22d51370b20"
geo_r = requests.get(freegeoip)
geo_j = json.loads(geo_r.text)
ip = geo_j['ip']
location = geo_j['city']
lat_ecp = geo_j['latitude']
lon_ecp = geo_j['longitude']
print("\nEmergency contact person details:")
print("Your location = " + location)


"""Calculate the distance between two points"""
try:
	lon_ecp, lat_ecp, user_longitude, user_latitude = map(radians, [lon_ecp, lat_ecp, user_longitude, user_latitude])

	# haversine formula 
	dlon = user_longitude - lon_ecp 
	dlat = user_latitude - lat_ecp 
	a = sin(dlat/2)**2 + cos(lat_ecp) * cos(user_latitude) * sin(dlon/2)**2
	c = 2 * asin(sqrt(a)) 
	km = 6367 * c

	km = ('%.0f'%km)

	msg_distance = 'Driver is about '+str(km)+' km away from you';
	print(msg_distance)
except:
	print('\nUnable to calculate distance.')
	msg_distance = 'Unable to calculate distance.'

website = 'https://www.google.com/maps/'
co_ordinates = "?q="+str(user_latitude)+","+str(user_longitude)


User/Driver details:
latitude = 80.00333002185538
longitude = 80.00333002185538
city = chennai

Emergency contact person details:
Your location = Spring Valley
Driver is about 7061 km away from you


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#login with your credentials
user_name = ""
user_pwd = ""

def send_mail(to, website, co_ordinates, msg_distance):
	# Create message container - the correct MIME type is multipart/alternative.
	msg = MIMEMultipart('alternative')
	msg['Subject'] = "About your driver"
	msg['From'] = "DDS App <" + user_name + ">"
	msg['To'] = to
	#print(msg['To'])

	# Create the body of the message (a plain-text and an HTML version).
	TEXT = "Hello,\n\nJust wanted to inform you that your driver detected drowsy.\n\nYour drive current location: "
	Disclaimer = "\n\nDisclaimer- This is an auto generated email, please do not reply to this email."
	BODY = TEXT + website + co_ordinates + "\n\n" + msg_distance + Disclaimer

	html = """\
	<html>
	  <head></head>
	  <body>
		<p><b>Hi!</b></p>
		<p>Just wanted to inform you that your driver detected drowsy.</p>
		<p>Your drive current location: <a href="https://www.google.com/maps/"""+ co_ordinates +"""">Google Map</a>.</p>
		<p>"""+ msg_distance +"""</p>
		<p><b>Disclaimer</b>- This is an auto generated email, please do not reply to this email.</p>
	  </body>
	</html>
	"""
	print ('\nEmail details:\nto = ' + to + '\nfrom = ' + user_name + '\nbody:\n' + BODY)

	part1 = MIMEText(BODY, 'plain')
	part2 = MIMEText(html, 'html')

	msg.attach(part1)
	msg.attach(part2)

	mail = smtplib.SMTP('smtp.gmail.com',587)
	#mail = smtplib.SMTP("smtp.mail.yahoo.com",587)
	mail.ehlo()
	mail.starttls()

	try:
		mail.login(user_name, user_pwd)
		mail.sendmail(msg['From'], msg['To'], msg.as_string())
		print("\nemail sent successfully")
	except smtplib.SMTPAuthenticationError:
		print ("\nInvalid user_name or password.")
	mail.close()

In [ ]:
pip install twilio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 7.4 MB/s 


In [ ]:
# Download the twilio-python library from twilio.com/docs/libraries/python
from twilio.rest import Client

# Find these values at https://twilio.com/user/account
account_sid = "AC********************************"
auth_token = ""

def send_sms(reciever, website, co_ordinates):
	client = Client(account_sid, auth_token)

	To = reciever
	Text = "\nYour driver detect drowsy.\nYour drive current location: " + website + co_ordinates
	print ('\nSMS details:\nto = ' + To + '\nfrom = +91 9150842667\nbody: ' + Text)

	client.api.account.messages.create(
		to = To,
		from_ = "+91 9150842667",
		body = Text)

	print ('\nSMS sent successfully.')